# Mortgage

In [1]:
import numpy as np
import pandas as pd
#import matplotlib.pyplot as plt
import plotly.graph_objects as go
from plotly.subplots import make_subplots

In [2]:
# =============================================================================
# A) BASE RATE MODEL: CIR (non-negative), exact transition sampling
#    dr_t = kappa*(theta - r_t) dt + sigma*sqrt(r_t) dW_t
# =============================================================================

def simulate_base_rate_cir_exact(
    n_sims: int,
    n_months: int,
    r0: float,
    kappa: float = 0.35,   # per year
    theta: float = 0.03,   # per year
    sigma: float = 0.10,   # per sqrt-year (CIR scale)
    cap: float | None = 0.20,
    seed: int | None = None
) -> np.ndarray:
    """
    Base-rate paths (decimal), shape (n_sims, n_months+1), monthly.
    Non-negative by construction (subject to floating point noise).
    """
    if r0 < 0:
        raise ValueError("r0 must be non-negative for CIR.")
    if kappa <= 0 or theta < 0 or sigma < 0:
        raise ValueError("kappa must be >0 and theta,sigma must be >=0.")

    rng = np.random.default_rng(seed)
    dt = 1.0 / 12.0

    rates = np.empty((n_sims, n_months + 1), dtype=float)
    rates[:, 0] = r0

    exp_kdt = np.exp(-kappa * dt)

    if sigma == 0.0:
        for t in range(n_months):
            rt = rates[:, t]
            r_next = theta + (rt - theta) * exp_kdt
            if cap is not None:
                r_next = np.minimum(r_next, cap)
            rates[:, t + 1] = r_next
        return rates

    # Exact CIR transition parameters
    c = (sigma**2 * (1.0 - exp_kdt)) / (4.0 * kappa)
    df = (4.0 * kappa * theta) / (sigma**2)
    denom = sigma**2 * (1.0 - exp_kdt)

    for t in range(n_months):
        rt = rates[:, t]
        nc = (4.0 * kappa * exp_kdt * rt) / denom
        x = rng.noncentral_chisquare(df=df, nonc=nc, size=n_sims)
        r_next = c * x
        if cap is not None:
            r_next = np.minimum(r_next, cap)
        rates[:, t + 1] = r_next

    # Clip tiny negative numerical noise
    rates = np.maximum(rates, 0.0)
    return rates


# =============================================================================
# B) PORTFOLIO MODEL: 60/40 with fat tails and correlation (monthly rebalanced)
#    Correlated multivariate Student-t shocks; monthly OCF drag.
# =============================================================================

def simulate_correlated_returns_t(
    n_sims: int,
    n_months: int,
    mus_annual: np.ndarray,      # (2,) equity, bond expected arithmetic returns
    sigmas_annual: np.ndarray,   # (2,) annual vols
    corr: float = -0.20,
    df: float = 8.0,
    seed: int | None = None
) -> np.ndarray:
    """
    Monthly log-returns for (equity, bond), shape (n_sims, n_months, 2).
    Uses multivariate t shocks standardized to unit marginal variance.
    """
    if df <= 2:
        raise ValueError("df must be > 2 for finite variance.")

    rng = np.random.default_rng(seed)
    dt = 1.0 / 12.0

    drift = (mus_annual - 0.5 * sigmas_annual**2) * dt
    vol = sigmas_annual * np.sqrt(dt)

    C = np.array([[1.0, corr], [corr, 1.0]], dtype=float)
    # Ensure PSD (small numerical guard)
    C = (C + C.T) / 2.0
    L = np.linalg.cholesky(C)

    Z = rng.standard_normal((n_sims, n_months, 2))
    Zc = Z @ L.T

    W = rng.chisquare(df, size=(n_sims, n_months, 1)) / df
    shocks = Zc / np.sqrt(W)

    # Standardize marginal variance of t(df) to 1: Var(t)=df/(df-2)
    shocks = shocks / np.sqrt(df / (df - 2.0))

    log_r = drift.reshape(1, 1, 2) + shocks * vol.reshape(1, 1, 2)
    return log_r


def simulate_60_40_portfolio_paths(
    n_sims: int,
    n_months: int,
    initial_value: float,
    w_equity: float = 0.60,
    equity_mu: float = 0.075,
    equity_sigma: float = 0.16,
    bond_mu: float = 0.035,
    bond_sigma: float = 0.07,
    eq_bond_corr: float = -0.20,
    t_df: float = 8.0,
    ocf_annual: float = 0.005,
    seed: int | None = None
) -> np.ndarray:
    """
    Portfolio values, shape (n_sims, n_months+1), monthly rebalanced to 60/40.
    Values are net of OCF drag applied monthly.
    """
    w_bond = 1.0 - w_equity
    if not (0.0 <= w_equity <= 1.0):
        raise ValueError("w_equity must be in [0,1].")

    log_r = simulate_correlated_returns_t(
        n_sims=n_sims,
        n_months=n_months,
        mus_annual=np.array([equity_mu, bond_mu]),
        sigmas_annual=np.array([equity_sigma, bond_sigma]),
        corr=eq_bond_corr,
        df=t_df,
        seed=seed
    )
    simple_r = np.expm1(log_r)

    values = np.empty((n_sims, n_months + 1), dtype=float)
    values[:, 0] = initial_value

    ocf_monthly_mult = 1.0 - (ocf_annual / 12.0)

    eq = values[:, 0] * w_equity
    bd = values[:, 0] * w_bond

    for t in range(n_months):
        eq *= (1.0 + simple_r[:, t, 0])
        bd *= (1.0 + simple_r[:, t, 1])
        total = (eq + bd) * ocf_monthly_mult

        # rebalance
        eq = total * w_equity
        bd = total * w_bond
        values[:, t + 1] = total

    return values


# =============================================================================
# C) MORTGAGE MODEL: repayment mortgage with monthly variable rate
#    Each month: compute payment that amortises over remaining term at current rate.
# =============================================================================

def level_payment(balance: np.ndarray, monthly_rate: np.ndarray, months_remaining: int) -> np.ndarray:
    r = monthly_rate
    n = months_remaining
    eps = 1e-12
    return np.where(
        np.abs(r) < eps,
        balance / n,
        balance * (r * (1.0 + r)**n) / ((1.0 + r)**n - 1.0)
    )


def simulate_mortgage_variable_rate(
    principal: float,
    annual_rates: np.ndarray,   # (n_sims, n_months) annual mortgage rate applied for each month
    term_months_remaining: int,
    n_months: int
):
    """
    Returns balances (n_sims,n_months+1), payments (n_sims,n_months),
    interest_paid, principal_paid.
    """
    n_sims = annual_rates.shape[0]
    balances = np.zeros((n_sims, n_months + 1), dtype=float)
    payments = np.zeros((n_sims, n_months), dtype=float)
    interest_paid = np.zeros((n_sims, n_months), dtype=float)
    principal_paid = np.zeros((n_sims, n_months), dtype=float)

    balances[:, 0] = principal

    for t in range(n_months):
        months_left = max(term_months_remaining - t, 1)
        bal = balances[:, t]
        m_r = annual_rates[:, t] / 12.0

        pmt = level_payment(bal, m_r, months_left)
        interest = bal * m_r
        principal_component = pmt - interest

        # prevent negative amortisation; cap to balance
        principal_component = np.maximum(principal_component, 0.0)
        principal_component = np.minimum(principal_component, bal)

        actual_pmt = interest + principal_component

        balances[:, t + 1] = bal - principal_component
        payments[:, t] = actual_pmt
        interest_paid[:, t] = interest
        principal_paid[:, t] = principal_component

    return balances, payments, interest_paid, principal_paid


# =============================================================================
# D) REFINANCE FEES
# =============================================================================

def refinance_fee_amount(balance: np.ndarray, fixed_fee: float, pct_fee: float) -> np.ndarray:
    return fixed_fee + pct_fee * balance


# =============================================================================
# E) RISK METRICS
# =============================================================================

def max_drawdown(path: np.ndarray) -> float:
    peak = np.maximum.accumulate(path)
    dd = (peak - path) / np.maximum(peak, 1e-12)
    return float(np.max(dd))


def summarize_distribution(x: np.ndarray) -> dict:
    return {
        "mean": float(np.mean(x)),
        "median": float(np.median(x)),
        "p05": float(np.quantile(x, 0.05)),
        "p25": float(np.quantile(x, 0.25)),
        "p75": float(np.quantile(x, 0.75)),
        "p95": float(np.quantile(x, 0.95)),
    }


# =============================================================================
# F) FULL SIMULATION (CIR rates + 60/40 + repayment mortgage + refi fees)
# =============================================================================

def simulate_invest_vs_repay(params: dict):
    """
    Strategies:
      - Repay today
      - Invest lump sum, keep mortgage

    cashflow_mode:
      - "exclude" (default): mortgage payments + refi fees are paid from income; wealth unaffected.
                             Repay strategy wealth = 0 baseline.
      - "include": mortgage payments + refi fees are paid from wealth (invest strategy).
                   Repay strategy invests the would-have-paid mortgage payment monthly.

    Returns:
      results_df: per-simulation end-horizon outcomes
      metrics_df: horizon-by-horizon metrics table
      paths: dict of arrays for further analysis/plotting
    """
    # ---- core settings
    n_sims = int(params.get("n_sims", 20000))
    seed = params.get("seed", 42)

    principal = float(params.get("mortgage_principal", 300_000.0))
    term_months = int(params.get("mortgage_term_months_remaining", 23 * 12 + 7))  # 23y7m default

    initial_cash = float(params.get("initial_cash", 300_000.0))

    # ---- CIR base rate parameters
    base_r0 = float(params.get("base_rate_r0", 0.05))     # set to current BoE base (decimal)
    kappa = float(params.get("base_kappa", 0.35))
    theta = float(params.get("base_theta", 0.03))
    sigma = float(params.get("base_sigma", 0.10))         # CIR sigma (try 0.06–0.15 range)
    cap = params.get("base_cap", 0.20)                    # can be None

    # ---- mortgage rate = base + margin
    margin = float(params.get("mortgage_margin_over_base", 0.0025))

    # ---- refinance
    refinance_every_months = int(params.get("refinance_every_months", 24))
    refinance_fixed_fee = float(params.get("refinance_fixed_fee", 0.0))
    refinance_pct_fee = float(params.get("refinance_pct_fee", 0.0))
    refinance_fee_capitalised = bool(params.get("refinance_fee_capitalised", False))

    # ---- investment parameters (GBP-ish nominal assumptions; editable)
    w_equity = float(params.get("w_equity", 0.60))
    equity_mu = float(params.get("equity_mu", 0.075))
    equity_sigma = float(params.get("equity_sigma", 0.16))
    bond_mu = float(params.get("bond_mu", 0.035))
    bond_sigma = float(params.get("bond_sigma", 0.07))
    eq_bond_corr = float(params.get("eq_bond_corr", -0.20))
    t_df = float(params.get("t_df", 8.0))
    ocf_annual = float(params.get("ocf_annual", 0.005))

    cashflow_mode = params.get("cashflow_mode", "exclude")
    if cashflow_mode not in ("exclude", "include"):
        raise ValueError("cashflow_mode must be 'exclude' or 'include'")

    horizons_years = params.get("horizons_years", [5, 10, 15, term_months / 12.0])
    horizons_months = sorted({int(round(h * 12)) for h in horizons_years if h > 0})
    horizons_months = [h for h in horizons_months if h <= term_months]
    if term_months not in horizons_months:
        horizons_months.append(term_months)

    n_months = term_months

    # ---- simulate base rates (CIR)
    base_rates = simulate_base_rate_cir_exact(
        n_sims=n_sims,
        n_months=n_months,
        r0=base_r0,
        kappa=kappa,
        theta=theta,
        sigma=sigma,
        cap=cap,
        seed=seed
    )

    # mortgage annual rate each month uses rate at start of month t
    mortgage_rates = base_rates[:, :-1] + margin  # (n_sims, n_months)

    # ---- simulate mortgage schedule (first pass)
    balances, payments, interest_paid, principal_paid = simulate_mortgage_variable_rate(
        principal=principal,
        annual_rates=mortgage_rates,
        term_months_remaining=term_months,
        n_months=n_months
    )

    # ---- refinance fee schedule (applied at month indices that are multiples of refinance_every_months)
    refi_months = set(range(refinance_every_months, n_months + 1, refinance_every_months))
    fees_at_month = np.zeros((n_sims, n_months + 1), dtype=float)

    for m in refi_months:
        fee_m = refinance_fee_amount(balances[:, m], refinance_fixed_fee, refinance_pct_fee)
        fees_at_month[:, m] = fee_m

    # If capitalising fees, approximate by adding them to balances from that point onward
    # and then rerunning the mortgage simulation to reflect higher balances.
    if refinance_fee_capitalised and (refinance_fixed_fee != 0.0 or refinance_pct_fee != 0.0):
        # Construct adjusted balances path by injecting fees (approx) and rerun schedule.
        # We inject fees to principal at the refinancing months by increasing principal dynamically is complex;
        # so we approximate by increasing principal up-front by PV of fees under expected path is not great.
        # Instead: do a pragmatic month-by-month rerun with fee injection inside loop (implemented below).
        balances, payments, interest_paid, principal_paid = simulate_mortgage_variable_rate_with_capitalised_fees(
            principal=principal,
            annual_rates=mortgage_rates,
            term_months_remaining=term_months,
            n_months=n_months,
            refinance_every_months=refinance_every_months,
            refinance_fixed_fee=refinance_fixed_fee,
            refinance_pct_fee=refinance_pct_fee
        )
        # recompute fees_at_month using the updated balances
        fees_at_month[:] = 0.0
        for m in refi_months:
            fees_at_month[:, m] = refinance_fee_amount(balances[:, m], refinance_fixed_fee, refinance_pct_fee)

    # ---- simulate investment portfolio lump-sum (net of OCF)
    invest_lump_paths = simulate_60_40_portfolio_paths(
        n_sims=n_sims,
        n_months=n_months,
        initial_value=initial_cash,
        w_equity=w_equity,
        equity_mu=equity_mu,
        equity_sigma=equity_sigma,
        bond_mu=bond_mu,
        bond_sigma=bond_sigma,
        eq_bond_corr=eq_bond_corr,
        t_df=t_df,
        ocf_annual=ocf_annual,
        seed=(seed + 1) if seed is not None else None
    )

    # growth factors implied by the path (already includes OCF and rebalancing)
    growth = invest_lump_paths[:, 1:] / np.maximum(invest_lump_paths[:, :-1], 1e-12)

    # ---- build strategy wealth paths depending on cashflow_mode
    if cashflow_mode == "exclude":
        # Invest strategy wealth is just the invested portfolio; mortgage paid from income.
        invest_wealth = invest_lump_paths.copy()

        # Repay strategy: baseline wealth = 0 (since cash used to pay off mortgage)
        repay_wealth = np.zeros_like(invest_wealth)

        # Advantage (wealth difference) vs repay is invest wealth minus remaining mortgage balance
        advantage_paths = invest_wealth - balances

    else:
        invest_wealth = np.zeros_like(invest_lump_paths)
        repay_wealth = np.zeros_like(invest_lump_paths)
        invest_wealth[:, 0] = initial_cash
        repay_wealth[:, 0] = 0.0  # cash used to repay today

        for t in range(n_months):
            # grow by portfolio factor
            invest_wealth[:, t + 1] = invest_wealth[:, t] * growth[:, t]
            repay_wealth[:, t + 1] = repay_wealth[:, t] * growth[:, t]

            # invest strategy: pay mortgage payment + (if not capitalised) refi fee out of wealth
            outflow = payments[:, t].copy()
            if ((t + 1) in refi_months) and (not refinance_fee_capitalised):
                outflow += fees_at_month[:, t + 1]

            invest_wealth[:, t + 1] = np.maximum(invest_wealth[:, t + 1] - outflow, 0.0)

            # repay strategy: invest the "saved payment" monthly
            repay_wealth[:, t + 1] += payments[:, t]

        advantage_paths = invest_wealth - balances

    # ---- metrics by horizon
    metrics_rows = []
    for H in horizons_months:
        adv = advantage_paths[:, H]
        underwater_months = (advantage_paths[:, :H + 1] < 0).sum(axis=1)

        mdd_invest = np.array([max_drawdown(invest_wealth[i, :H + 1]) for i in range(n_sims)])

        metrics = {
            "horizon_years": H / 12.0,
            "prob_underperform_vs_repay": float(np.mean(adv < 0)),
            "advantage_p05": float(np.quantile(adv, 0.05)),
            "advantage_median": float(np.median(adv)),
            "advantage_p95": float(np.quantile(adv, 0.95)),
            "max_drawdown_invest_p95": float(np.quantile(mdd_invest, 0.95)),
            "time_underwater_months_median": float(np.median(underwater_months)),
            "time_underwater_months_p90": float(np.quantile(underwater_months, 0.90)),
            "mortgage_balance_median": float(np.median(balances[:, H])),
            "invest_wealth_median": float(np.median(invest_wealth[:, H])),
            "repay_wealth_median": float(np.median(repay_wealth[:, H])),
        }
        metrics_rows.append(metrics)

    metrics_df = pd.DataFrame(metrics_rows).sort_values("horizon_years").reset_index(drop=True)

    # ---- end-horizon results table
    endH = n_months
    results_df = pd.DataFrame({
        "advantage_vs_repay_end": advantage_paths[:, endH],
        "invest_wealth_end": invest_wealth[:, endH],
        "repay_wealth_end": repay_wealth[:, endH],
        "mortgage_balance_end": balances[:, endH],
        "base_rate_end": base_rates[:, endH],
    })
    results_df["max_drawdown_invest"] = np.array([max_drawdown(invest_wealth[i, :endH + 1]) for i in range(n_sims)])
    results_df["underwater_months_total"] = (advantage_paths[:, :endH + 1] < 0).sum(axis=1)

    paths = {
        "base_rates": base_rates,
        "mortgage_rates": mortgage_rates,
        "balances": balances,
        "payments": payments,
        "fees_at_month": fees_at_month,
        "invest_wealth": invest_wealth,
        "repay_wealth": repay_wealth,
        "advantage_paths": advantage_paths,
    }

    return results_df, metrics_df, paths


# =============================================================================
# G) OPTIONAL: more faithful capitalised-fee mortgage simulation
#    (injects fee into balance on refi months before computing that month's payment)
# =============================================================================

def simulate_mortgage_variable_rate_with_capitalised_fees(
    principal: float,
    annual_rates: np.ndarray,   # (n_sims, n_months)
    term_months_remaining: int,
    n_months: int,
    refinance_every_months: int,
    refinance_fixed_fee: float,
    refinance_pct_fee: float,
):
    n_sims = annual_rates.shape[0]
    balances = np.zeros((n_sims, n_months + 1), dtype=float)
    payments = np.zeros((n_sims, n_months), dtype=float)
    interest_paid = np.zeros((n_sims, n_months), dtype=float)
    principal_paid = np.zeros((n_sims, n_months), dtype=float)

    balances[:, 0] = principal

    for t in range(n_months):
        months_left = max(term_months_remaining - t, 1)
        bal = balances[:, t].copy()

        # Inject fee at the start of the refinancing month (t+1 in month-counting terms)
        # If you interpret refinance as occurring at month 24, 48, ... then apply when (t+1) % 24 == 0
        if refinance_every_months > 0 and ((t + 1) % refinance_every_months == 0):
            fee = refinance_fee_amount(bal, refinance_fixed_fee, refinance_pct_fee)
            bal = bal + fee  # capitalise fee into mortgage principal

        m_r = annual_rates[:, t] / 12.0
        pmt = level_payment(bal, m_r, months_left)

        interest = bal * m_r
        principal_component = pmt - interest
        principal_component = np.maximum(principal_component, 0.0)
        principal_component = np.minimum(principal_component, bal)

        actual_pmt = interest + principal_component

        balances[:, t + 1] = bal - principal_component
        payments[:, t] = actual_pmt
        interest_paid[:, t] = interest
        principal_paid[:, t] = principal_component

    return balances, payments, interest_paid, principal_paid


# =============================================================================
# H) PLOTTING HELPERS
# =============================================================================

# -----------------------------
# Horizon marker helpers
# -----------------------------
def _horizon_months(params):
    hs = params.get("horizons_years", [])
    hm = sorted({int(round(h * 12)) for h in hs if h and h > 0})
    return hm

def _add_horizon_vlines(fig, horizon_months, yref="paper", row=None, col=None):
    """
    Adds vertical lines and labels at horizon months.
    yref defaults to paper so the line spans the plot height.
    """
    for m in horizon_months:
        fig.add_vline(
            x=m,
            line_width=1,
            line_dash="dot",
            row=row, col=col
        )
        # optional annotation
        fig.add_annotation(
            x=m, y=1.02, xref="x", yref=yref,
            text=f"{m/12:.1f}y",
            showarrow=False,
            yanchor="bottom",
            font=dict(size=10),
            row=row, col=col
        )

# -----------------------------
# Plot builders
# -----------------------------
def interactive_spaghetti(paths_2d, title, y_label, params, n_paths=60):
    horizon_months = _horizon_months(params)
    n = min(n_paths, paths_2d.shape[0])
    T = paths_2d.shape[1]
    x = np.arange(T)

    fig = go.Figure()
    for i in range(n):
        fig.add_trace(go.Scatter(x=x, y=paths_2d[i, :], mode="lines", line=dict(width=1), name=f"path {i+1}", showlegend=False))

    _add_horizon_vlines(fig, horizon_months)

    fig.update_layout(
        title=title,
        xaxis_title="Month",
        yaxis_title=y_label,
        hovermode="x unified",
        height=450
    )
    fig.show()


def interactive_fan(paths_2d, title, y_label, params, percentiles=(5, 25, 50, 75, 95), shade_iqr=True):
    horizon_months = _horizon_months(params)
    T = paths_2d.shape[1]
    x = np.arange(T)

    p = np.percentile(paths_2d, percentiles, axis=0)

    fig = go.Figure()

    if shade_iqr:
        p25 = np.percentile(paths_2d, 25, axis=0)
        p75 = np.percentile(paths_2d, 75, axis=0)
        # Fill between p25 and p75
        fig.add_trace(go.Scatter(x=x, y=p75, mode="lines", line=dict(width=0), showlegend=False))
        fig.add_trace(go.Scatter(x=x, y=p25, mode="lines", line=dict(width=0), fill="tonexty", name="IQR (25–75)"))

    for i, pct in enumerate(percentiles):
        fig.add_trace(go.Scatter(x=x, y=p[i, :], mode="lines", line=dict(width=2 if pct == 50 else 1), name=f"p{pct:02d}"))

    _add_horizon_vlines(fig, horizon_months)

    fig.update_layout(
        title=title,
        xaxis_title="Month",
        yaxis_title=y_label,
        hovermode="x unified",
        height=450
    )
    fig.show()


def interactive_advantage_fan(adv_paths, title, params):
    # Fan chart + zero line
    horizon_months = _horizon_months(params)
    T = adv_paths.shape[1]
    x = np.arange(T)

    p = np.percentile(adv_paths, [5, 25, 50, 75, 95], axis=0)
    p05, p25, p50, p75, p95 = p

    fig = go.Figure()

    # IQR band
    fig.add_trace(go.Scatter(x=x, y=p75, mode="lines", line=dict(width=0), showlegend=False))
    fig.add_trace(go.Scatter(x=x, y=p25, mode="lines", line=dict(width=0), fill="tonexty", name="IQR (25–75)"))

    # percentile lines
    fig.add_trace(go.Scatter(x=x, y=p05, mode="lines", line=dict(width=1), name="p05"))
    fig.add_trace(go.Scatter(x=x, y=p50, mode="lines", line=dict(width=2), name="p50 (median)"))
    fig.add_trace(go.Scatter(x=x, y=p95, mode="lines", line=dict(width=1), name="p95"))

    # zero line
    fig.add_hline(y=0.0, line_width=1)

    _add_horizon_vlines(fig, horizon_months)

    fig.update_layout(
        title=title,
        xaxis_title="Month",
        yaxis_title="£ Advantage (Wealth - Mortgage Balance)",
        hovermode="x unified",
        height=450
    )
    fig.show()


def interactive_underwater_probability(adv_paths, title, params):
    horizon_months = _horizon_months(params)
    prob = (adv_paths < 0).mean(axis=0)
    x = np.arange(prob.shape[0])

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=x, y=prob, mode="lines", line=dict(width=2), name="P(Adv < 0)"))

    _add_horizon_vlines(fig, horizon_months)

    fig.update_layout(
        title=title,
        xaxis_title="Month",
        yaxis_title="Probability",
        yaxis=dict(range=[0, 1]),
        hovermode="x unified",
        height=450
    )
    fig.show()


def interactive_histogram(series, title, x_label, params, bins=80):
    """
    Histogram with optional vertical lines at 0 (for advantage).
    Horizon markers are not meaningful on a histogram, so not included here.
    """
    x = np.asarray(series)
    fig = go.Figure()
    fig.add_trace(go.Histogram(x=x, nbinsx=bins, name=""))
    fig.update_layout(
        title=title,
        xaxis_title=x_label,
        yaxis_title="Count",
        height=450
    )
    fig.show()


def interactive_dashboard(paths, results_df, params, n_paths=30):
    """
    Compact interactive dashboard: 2x2 fan charts + underwater probability.
    """
    horizon_months = _horizon_months(params)

    fig = make_subplots(
        rows=3, cols=2,
        subplot_titles=(
            "BoE Base Rate (Fan)", "Mortgage Balance (Fan)",
            "Invest Wealth (Fan)", "Advantage (Fan + 0 line)",
            "P(Advantage < 0) Over Time", "Advantage at Term End (Histogram)"
        ),
        vertical_spacing=0.10
    )

    def add_fan_to_subplot(paths_2d, row, col, y_title, include_zero=False):
        T = paths_2d.shape[1]
        x = np.arange(T)
        p05, p25, p50, p75, p95 = np.percentile(paths_2d, [5, 25, 50, 75, 95], axis=0)

        # IQR band
        fig.add_trace(go.Scatter(x=x, y=p75, mode="lines", line=dict(width=0), showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=x, y=p25, mode="lines", line=dict(width=0), fill="tonexty", name="IQR (25–75)", showlegend=False), row=row, col=col)

        # lines
        fig.add_trace(go.Scatter(x=x, y=p05, mode="lines", line=dict(width=1), name="p05", showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=x, y=p50, mode="lines", line=dict(width=2), name="p50", showlegend=False), row=row, col=col)
        fig.add_trace(go.Scatter(x=x, y=p95, mode="lines", line=dict(width=1), name="p95", showlegend=False), row=row, col=col)

        # Horizon vlines
        for m in horizon_months:
            fig.add_vline(x=m, line_width=1, line_dash="dot", row=row, col=col)

        if include_zero:
            fig.add_hline(y=0.0, line_width=1, row=row, col=col)

        fig.update_yaxes(title_text=y_title, row=row, col=col)

    add_fan_to_subplot(paths["base_rates"], row=1, col=1, y_title="Rate (dec.)")
    add_fan_to_subplot(paths["balances"], row=1, col=2, y_title="£ Balance")
    add_fan_to_subplot(paths["invest_wealth"], row=2, col=1, y_title="£ Wealth")
    add_fan_to_subplot(paths["advantage_paths"], row=2, col=2, y_title="£ Advantage", include_zero=True)

    # Underwater probability
    prob = (paths["advantage_paths"] < 0).mean(axis=0)
    x = np.arange(prob.shape[0])
    fig.add_trace(go.Scatter(x=x, y=prob, mode="lines", line=dict(width=2), showlegend=False), row=3, col=1)
    for m in horizon_months:
        fig.add_vline(x=m, line_width=1, line_dash="dot", row=3, col=1)
    fig.update_yaxes(title_text="Probability", range=[0, 1], row=3, col=1)

    # End histogram
    adv_end = results_df["advantage_vs_repay_end"].to_numpy()
    fig.add_trace(go.Histogram(x=adv_end, nbinsx=80, showlegend=False), row=3, col=2)
    fig.update_xaxes(title_text="£ Advantage", row=3, col=2)
    fig.update_yaxes(title_text="Count", row=3, col=2)

    fig.update_xaxes(title_text="Month", row=1, col=1)
    fig.update_xaxes(title_text="Month", row=1, col=2)
    fig.update_xaxes(title_text="Month", row=2, col=1)
    fig.update_xaxes(title_text="Month", row=2, col=2)
    fig.update_xaxes(title_text="Month", row=3, col=1)

    fig.update_layout(height=1100, hovermode="x unified", title="Mortgage vs Invest Simulation Dashboard")
    fig.show()



In [3]:
# =============================================================================
# I) EXAMPLE RUN: your specifics (23y7m, margin default 0.25%, 60/40, OCF)
# =============================================================================

params = {
    "n_sims": 20_000,
    "seed": 42,

    # Mortgage
    "mortgage_principal": 300_000.0,
    "mortgage_term_months_remaining": 23 * 12 + 7,  # 23y7m
    "mortgage_margin_over_base": 0.0025,            # 0.25%

    # CIR Base rate model (set r0 to current BoE base rate in decimal)
    "base_rate_r0": 0.05,
    "base_kappa": 0.50,   # faster mean reversion often looks more realistic for policy rates
    "base_theta": 0.03,
    "base_sigma": 0.10,   # CIR scale; try 0.06–0.15
    "base_cap": 0.20,

    # Refinance
    "refinance_every_months": 24,
    "refinance_fixed_fee": 999.0,  # set 0 if you want
    "refinance_pct_fee": 0.0,
    "refinance_fee_capitalised": False,

    # Investment (GBP-ish nominal assumptions; adjust to taste)
    "initial_cash": 300_000.0,
    "w_equity": 0.60,
    "equity_mu": 0.075,
    "equity_sigma": 0.16,
    "bond_mu": 0.035,
    "bond_sigma": 0.07,
    "eq_bond_corr": -0.20,
    "t_df": 8.0,
    "ocf_annual": 0.005,  # 0.5% p.a.

    # Cashflow treatment
    "cashflow_mode": "exclude",  # default per your request

    # Horizons (years)
    "horizons_years": [1, 3, 5, 7, 10, 15, 23 + 7/12],
}

results_df, metrics_df, paths = simulate_invest_vs_repay(params)

# View horizon metrics
metrics_df


,horizon_years,prob_underperform_vs_repay,advantage_p05,advantage_median,advantage_p95,max_drawdown_invest_p95,time_underwater_months_median,time_underwater_months_p90,mortgage_balance_median,invest_wealth_median,repay_wealth_median
0,1.000000,0.21325,-23415.350981,22435.251687,7.558500e+04,0.139064,2.0,10.0,292922.894076,315352.414919,0.0
1,3.000000,0.07915,-10301.113217,71808.441883,1.802791e+05,0.215078,3.0,21.0,275822.735371,347948.079748,0.0
2,5.000000,0.03120,12518.955278,127808.507345,2.903508e+05,0.252506,3.0,24.0,256317.554963,384374.640784,0.0
3,7.000000,0.00965,43959.831014,189245.811681,4.069195e+05,0.279001,3.0,25.0,235323.178704,425286.060054,0.0
4,10.000000,0.00125,96663.172245,290913.680959,6.081609e+05,0.302953,3.0,25.0,201236.352060,493660.466584,0.0
5,15.000000,0.00000,206216.326929,495239.966269,1.022885e+06,0.330030,3.0,25.0,136902.169434,632924.582032,0.0
6,23.583333,0.00000,450495.408772,966288.259279,2.068106e+06,0.359406,3.0,25.0,0.000000,966288.259279,0.0


What you should edit first

params["base_rate_r0"]: set to the current BoE base rate in decimal (e.g., 0.0525 for 5.25%).

params["base_sigma"]: CIR vol scale; if paths look too flat, increase it (e.g., 0.12–0.15). If too wild, decrease (e.g., 0.06–0.08).

params["refinance_fixed_fee"] / params["refinance_pct_fee"] / params["refinance_every_months"]: to match your actual remortgage process.

params["cashflow_mode"]: keep "exclude" for your default assumption; switch to "include" to model mortgage payments coming out of investable wealth (and the repay strategy investing the saved payments).

In [4]:
results_df

,advantage_vs_repay_end,invest_wealth_end,repay_wealth_end,mortgage_balance_end,base_rate_end,max_drawdown_invest,underwater_months_total
0,6.995995e+05,6.995995e+05,0.0,0.000000e+00,0.022963,0.183691,26
1,9.071269e+05,9.071269e+05,0.0,3.797140e-11,0.040471,0.129983,16
2,1.422661e+06,1.422661e+06,0.0,9.549694e-12,0.041191,0.166054,1
3,1.511314e+06,1.511314e+06,0.0,9.163159e-11,0.013983,0.182850,0
4,9.486770e+05,9.486770e+05,0.0,4.206413e-11,0.017371,0.241507,0
...,...,...,...,...,...,...,...
19995,1.512169e+06,1.512169e+06,0.0,0.000000e+00,0.024811,0.235468,2
19996,2.963343e+05,2.963343e+05,0.0,0.000000e+00,0.004527,0.314694,26
19997,1.655604e+06,1.655604e+06,0.0,2.751221e-11,0.019490,0.129134,5
19998,5.579439e+05,5.579439e+05,0.0,1.773515e-11,0.015145,0.285075,14


In [5]:
metrics_df

,horizon_years,prob_underperform_vs_repay,advantage_p05,advantage_median,advantage_p95,max_drawdown_invest_p95,time_underwater_months_median,time_underwater_months_p90,mortgage_balance_median,invest_wealth_median,repay_wealth_median
0,1.000000,0.21325,-23415.350981,22435.251687,7.558500e+04,0.139064,2.0,10.0,292922.894076,315352.414919,0.0
1,3.000000,0.07915,-10301.113217,71808.441883,1.802791e+05,0.215078,3.0,21.0,275822.735371,347948.079748,0.0
2,5.000000,0.03120,12518.955278,127808.507345,2.903508e+05,0.252506,3.0,24.0,256317.554963,384374.640784,0.0
3,7.000000,0.00965,43959.831014,189245.811681,4.069195e+05,0.279001,3.0,25.0,235323.178704,425286.060054,0.0
4,10.000000,0.00125,96663.172245,290913.680959,6.081609e+05,0.302953,3.0,25.0,201236.352060,493660.466584,0.0
5,15.000000,0.00000,206216.326929,495239.966269,1.022885e+06,0.330030,3.0,25.0,136902.169434,632924.582032,0.0
6,23.583333,0.00000,450495.408772,966288.259279,2.068106e+06,0.359406,3.0,25.0,0.000000,966288.259279,0.0


In [ ]:
# -----------------------------
# Usage examples
# -----------------------------
# 1) Individual interactive charts
interactive_spaghetti(paths["base_rates"], "Sample BoE Base Rate Paths (CIR)", "Base rate (decimal)", params, n_paths=40)
interactive_fan(paths["base_rates"], "BoE Base Rate Fan Chart", "Base rate (decimal)", params)

interactive_fan(paths["balances"], "Mortgage Balance Fan Chart", "£ Mortgage balance", params)
interactive_fan(paths["invest_wealth"], "Invest Wealth Fan Chart", "£ Wealth", params)

interactive_advantage_fan(paths["advantage_paths"], "Advantage Fan Chart (Wealth - Balance)", params)
interactive_underwater_probability(paths["advantage_paths"], "Probability(Advantage < 0) Over Time", params)

interactive_histogram(results_df["advantage_vs_repay_end"], "Advantage at Term End", "£ Advantage", params)
interactive_histogram(results_df["max_drawdown_invest"], "Max Drawdown (Invest Wealth) Over Full Horizon", "Max drawdown (fraction)", params)

# 2) One-page interactive dashboard
interactive_dashboard(paths, results_df, params, n_paths=30)

In [6]:
interactive_dashboard(paths, results_df, params, n_paths=30)